# Preprocessing 05 - event detection

## Import Libraries

In [7]:
import pandas as pd
import numpy as np
from dataEvaluation.utils.remodnav import perform_remodnav
import scipy.signal as signal
import os
import contextlib
from tqdm.notebook import tqdm

## Eyetracking Event Detection

In [8]:
def count_nan_beginning(data):
    count = 0
    for value in data:
        if np.isnan(value):
            count += 1
        else:
            return count
    return count

def count_nan_end(data):
    count = 0
    for value in data[::-1]:
        if np.isnan(value):
            count += 1
        else:
            return count
    return count

In [9]:
# Read in the Behavioral Data
df_behavioral = pd.read_csv("./data/filteredData/filtered_data.csv")

sampling_rate = 250.0
screen_resolution = (1920, 1080)
screen_size = (56.0, 31.5)
screen_distance = 60.0
x_res = 1920.0
y_res = 1080.0

df_events = pd.DataFrame(columns=["Participant", "Algorithm",
                                  "id", "label", "start_x", "start_y", "end_x", "end_y", "start_time", "end_time",
                                  "amp", "peak_vel", "med_vel", "avg_vel"])
errors = []

for index, row in tqdm(df_behavioral.iterrows(), total=len(df_behavioral)):
    # read in eyetracking file
    df_eyetracking = pd.read_csv(row["Eyetracking"])
    participant = row["Participant"]
    algorithm = row["Algorithm"]

    # normalize the time regarding eyetracking to 0
    df_eyetracking["time"] = df_eyetracking["time"].astype(float)
    df_eyetracking["time"] = df_eyetracking["time"] - df_eyetracking["time"].iloc[0]

    # drop unused columns
    df_eyetracking = df_eyetracking.drop(columns=["l_gaze_point_in_user_coordinate_system_x",
                                                  "l_gaze_point_in_user_coordinate_system_y",
                                                  "l_gaze_point_in_user_coordinate_system_z",
                                                  "r_gaze_point_in_user_coordinate_system_x",
                                                  "r_gaze_point_in_user_coordinate_system_y",
                                                  "r_gaze_point_in_user_coordinate_system_z",
                                                  "l_gaze_origin_in_user_coordinate_system_x",
                                                  "l_gaze_origin_in_user_coordinate_system_y",
                                                  "l_gaze_origin_in_user_coordinate_system_z",
                                                  "r_gaze_origin_in_user_coordinate_system_x",
                                                  "r_gaze_origin_in_user_coordinate_system_y",
                                                  "r_gaze_origin_in_user_coordinate_system_z"])

    # convert eyetracking data to display coordinates
    df_eyetracking["l_display_x"] = df_eyetracking["l_display_x"].astype(float) * x_res
    df_eyetracking["l_display_y"] = df_eyetracking["l_display_y"].astype(float) * y_res
    df_eyetracking["r_display_x"] = df_eyetracking["r_display_x"].astype(float) * x_res
    df_eyetracking["r_display_y"] = df_eyetracking["r_display_y"].astype(float) * y_res

    # convert eyetracking data to I2MC valid flags
    df_eyetracking["l_valid"] = df_eyetracking["l_valid"].astype(int)
    df_eyetracking["r_valid"] = df_eyetracking["r_valid"].astype(int)

    # convert miss column to right integer used by I2MC
    df_eyetracking["l_miss_x"] = df_eyetracking.apply(lambda row: row["l_display_x"] < -x_res or row["l_display_x"] > 2 * x_res, axis=1)
    df_eyetracking["l_miss_y"] = df_eyetracking.apply(lambda row: row["l_display_y"] < -y_res or row["l_display_y"] > 2 * y_res, axis=1)
    df_eyetracking["r_miss_x"] = df_eyetracking.apply(lambda row: row["r_display_x"] < -x_res or row["r_display_x"] > 2 * x_res, axis=1)
    df_eyetracking["r_miss_y"] = df_eyetracking.apply(lambda row: row["r_display_y"] < -y_res or row["r_display_y"] > 2 * y_res, axis=1)

    df_eyetracking["l_miss"] = df_eyetracking.apply(lambda row: row["l_miss_x"] or row["l_miss_y"] or not row["l_valid"] >= 1, axis=1)
    df_eyetracking["r_miss"] = df_eyetracking.apply(lambda row: row["r_miss_x"] or row["r_miss_y"] or not row["r_valid"] >= 1, axis=1)

    # Set a default value for missing data
    df_eyetracking.loc[df_eyetracking["l_miss"], "l_display_x"] = np.nan
    df_eyetracking.loc[df_eyetracking["l_miss"], "l_display_y"] = np.nan
    df_eyetracking.loc[df_eyetracking["r_miss"], "r_display_x"] = np.nan
    df_eyetracking.loc[df_eyetracking["r_miss"], "r_display_y"] = np.nan

    # interpolate missing data
    try:
        l_display_x = df_eyetracking["l_display_x"].interpolate('cubic')
        l_display_y = df_eyetracking["l_display_y"].interpolate('cubic')
    except:
        l_display_x = df_eyetracking["l_display_x"]
        l_display_y = df_eyetracking["l_display_y"]

    try:
        r_display_x = df_eyetracking["r_display_x"].interpolate('cubic')
        r_display_y = df_eyetracking["r_display_y"].interpolate('cubic')
    except:
        r_display_x = df_eyetracking["r_display_x"]
        r_display_y = df_eyetracking["r_display_y"]


    with open(os.devnull, 'w') as devnull:
        with contextlib.redirect_stdout(devnull):
            with contextlib.redirect_stderr(devnull):
                # average x of both eyes
                avg_x = np.nanmean(np.stack([l_display_x, r_display_x]), axis=0)
                # average y of both eyes
                avg_y = np.nanmean(np.stack([l_display_y, r_display_y]), axis=0)

    # apply moving average filter
    avg_x = signal.medfilt(avg_x, kernel_size=7)
    avg_y = signal.medfilt(avg_y, kernel_size=7)

    nan_beginning = max(count_nan_beginning(avg_x), count_nan_beginning(avg_y))
    nan_end = max(count_nan_end(avg_x), count_nan_end(avg_y), 1)

    avg_x = avg_x[nan_beginning:-nan_end]
    avg_y = avg_y[nan_beginning:-nan_end]

    time_offset = 1.0/sampling_rate * nan_beginning

    try:
        # disallow outputting for next function
        with open(os.devnull, 'w') as devnull:
            with contextlib.redirect_stdout(devnull):
                with contextlib.redirect_stderr(devnull):
                    events, pp, clf = perform_remodnav(
                        avg_x, avg_y,
                        sampling_rate,
                        screen_width=screen_size[0],
                        screen_width_pixels=screen_resolution[0],
                        screen_distance=screen_distance,
                        savgol_length=0.02)
    except Exception as e:
        print(index)
        print(e)

    df_tmp = pd.DataFrame(columns=df_events.columns)
    for cure_idx, value in enumerate(events):
        value["Participant"] = participant
        value["Algorithm"] = algorithm
        value["start_time"] += time_offset
        value["end_time"] += time_offset
        df_tmp = df_tmp.append(value, ignore_index=True)

    df_tmp = df_tmp.reset_index(drop=True)
    # append the non duplicated rows to the final dataframe
    df_events = df_events.append(df_tmp)
    df_events = df_events.reset_index(drop=True)

  0%|          | 0/1072 [00:00<?, ?it/s]

## Export The Data

In [11]:
for (participant, algorithm), df_group in tqdm(df_events.groupby(["Participant", "Algorithm"])):
    df_group = df_group.reset_index(drop=True)
    df_group = df_group.sort_values(by="start_time")
    # drop columns that are not needed
    df_group = df_group.drop(columns=["Participant", "Algorithm"])
    df_group.to_csv(f"./data/filteredData/Participant{str(participant).zfill(2)}/{algorithm}_eyetracking.csv")

  0%|          | 0/1072 [00:00<?, ?it/s]

36 ReverseArray
41 CheckIfLettersOnly
60 ArrayAverage
